Webpage dataset: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=form

In [1]:
"""
Author: Sergi Mas Pujol
Last update: 13/06/2021

Python version: 3.6

Notes:
    1. To use the API you must have configured the CDS API key and the CDS API client.
        For further infomraiton, go to: https://cds.climate.copernicus.eu/api-how-to
    2. You must have unique_traffic_volumes_regulations -> A list (array) with the names of the TV you
        want to extract the weather infroamtion
"""

'\nAutohr: Sergi Mas Pujol\nLast update: 04/12/2020\n\nPython version: 3.6\n'

In [2]:
import cdsapi
c = cdsapi.Client()

import numpy as np
import time as t

from utils_airspaceConfiguration_aircraftCharacteristics import extractNames_primitiveSectors_fromRNESTname, \
                                                                extract_coordinates_primitive_sectors, \
                                                                compute_central_coordinates_sector, \
                                                                compute_coordinates_subregion_extraction_weather_features

# Set the main features for the extraction

In [3]:
unique_traffic_volumes_regulations = np.load('./Export_weather_information/unique_list_volumes_with_weather_regulations.npy')

In [4]:
unique_traffic_volumes_regulations

array(['B3OL', 'DWST', 'DHOL', 'B3LH', 'BWST', 'B3EH', 'HWST', 'H5WH',
       'HSOL', 'D6WH', 'BALB', 'B5WH', 'D5JH', 'HCEL', 'H5RL', 'B5WL',
       'B5KL', 'HEST', 'BLUX', 'BOLN', 'D3WL', 'D5JL', 'HRHR', 'B3LL',
       'DEST', 'BEST', 'DJEV', 'D3WLC', 'HMNS', 'B5NL', 'H5ML', 'D3WM'],
      dtype='<U5')

In [11]:
variables = ['divergence', 'fraction_of_cloud_cover', 'geopotential',
            'ozone_mass_mixing_ratio', 'potential_vorticity', 'relative_humidity',
            'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 
             'specific_humidity', 'specific_rain_water_content', 
             'specific_snow_water_content', 'temperature', 'u_component_of_wind', 
             'v_component_of_wind', 'vertical_velocity', 'vorticity',]

# variables = ['divergence', 'fraction_of_cloud_cover', 'geopotential', 
#              'potential_vorticity', 'relative_humidity',
#              'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 
#              'specific_humidity', 'specific_rain_water_content', 
#              'specific_snow_water_content', 'temperature',
#              'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity',
#              'vorticity',]


pressure_level = '250'

year = '2019'

months = ['06', '07', '08', '09', ]

days = ['01', '02', '03',
       '04', '05', '06',
       '07', '08', '09',
       '10', '11', '12',
       '13', '14', '15',
       '16', '17', '18',
       '19', '20', '21',
       '22', '23', '24',
       '25', '26', '27',
       '28', '29', '30', '31',]

times = ['00:00', '01:00', '02:00',
        '03:00', '04:00', '05:00',
        '06:00', '07:00', '08:00',
        '09:00', '10:00', '11:00',
        '12:00', '13:00', '14:00',
        '15:00', '16:00', '17:00',
        '18:00', '19:00', '20:00',
        '21:00', '22:00', '23:00',]

In [12]:
len(variables)

16

# Extract the features

In [22]:
# Search index for a specific volume
sector = 'D3WLC'
index = np.where(unique_traffic_volumes_regulations == sector)[0][0]
print(index)
print(unique_traffic_volumes_regulations[index])

27
D3WLC


In [23]:
for volume in unique_traffic_volumes_regulations[11:]:
    t0_volume = t.time()
    for month in months:
        t0_month = t.time()
        print(volume)
        print(month)

        # Extract the coordinated for the sub-region related to the TV
        primitiveSectors, volumes = extractNames_primitiveSectors_fromRNESTname(volume)
        lons_primitiveRegions, lats_primitiveRegions = extract_coordinates_primitive_sectors(primitiveSectors, volumes)
        lon_centre, lat_centre = compute_central_coordinates_sector(lons_primitiveRegions, lats_primitiveRegions)
        [north, west, south, east] = compute_coordinates_subregion_extraction_weather_features(lon_centre, 
                                                                                           lat_centre, 
                                                                                           width_region_meters=200000, 
                                                                                           height_region_meters=200000)
    
        print([north, west, south, east])
        # Extract the weather features
        c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'format': 'grib',
            'area': [north, west, south, east],
            'variable': variables,
            'pressure_level': pressure_level,
            'year': year,
            'month': month,
            'day': days,
            'time': times,
        },
        './Export_weather_information/'+volume+'_'+month+'_'+pressure_level+'.grib')
#         '../'+volume+'_'+month+'_'+pressure_level+'.grib')
        
        t1_month = t.time()
        print('Time previous month: {time}'.format(time=t1_month-t0_month))
        
    t1_volume = t.time()
    print('Time previous TV: {time}'.format(time=t1_volume-t0_volume))

D3WLC
06
[52.95, 2.97, 50.95, 4.970000000000001]
D3WLC
07
[52.95, 2.97, 50.95, 4.970000000000001]
D3WLC
08
[52.95, 2.97, 50.95, 4.970000000000001]
D3WLC
09
[52.95, 2.97, 50.95, 4.970000000000001]
